In [ ]:
!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [ ]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [ ]:
dataset_folder = Path.cwd().joinpath("sample_data")
dataset_path = dataset_folder.joinpath('MELD_train_efr.json')

df = pd.read_json(dataset_path)

# Assuming 'utterances' column is a list of strings
df['utterances'] = df['utterances'].apply(lambda x: ' '.join(x))
# Group by 'episode' and concatenate utterances with '[SEP]' separator
df['bert_input'] = df.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))

from sklearn.model_selection import GroupShuffleSplit

# Define the splitter based on the 'episode' column
gss = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)

# Execute the split
for train_idx, test_idx in gss.split(df, groups=df['episode']):
    train_data = df.iloc[train_idx]
    test_data = df.iloc[test_idx]

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_data['labels_column'])

print(train_data)
print(test_data)

KeyError: 'labels_column'

In [ ]:
dataset_folder = Path.cwd().joinpath("sample_data")
dataset_path = dataset_folder.joinpath('MELD_train_efr.json')

df = pd.read_json(dataset_path)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [ ]:
# import pandas as pd
# from sklearn.model_selection import GroupShuffleSplit

# # Create a group k-fold splitter
# gss = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)

# # Assume 'episode' is the column that contains dialogue or episode IDs
# for train_idx, test_idx in gss.split(df, groups=df['episode']):
#     train_data = df.iloc[train_idx]
#     test_data = df.iloc[test_idx]

In [ ]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# Assuming 'utterances' column is a list of strings
df['utterances'] = df['utterances'].apply(lambda x: ' '.join(x))
# Group by 'episode' and concatenate utterances with '[SEP]' separator
df['bert_input'] = df.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

# Define the splitter based on the 'episode' column
gss = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)

# Execute the split
for train_idx, test_idx in gss.split(df, groups=df['episode']):
    train_data = df.iloc[train_idx]
    test_data = df.iloc[test_idx]

# Repeat for creating a validation set if needed

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# mlb = MultiLabelBinarizer()
# train_labels = mlb.fit_transform(train_data['labels_column'])

KeyError: 'labels_column'

In [ ]:
# Repeat the above splitting process to create validation set from train_data if needed

# Concatenate utterances in each dialogue with [SEP] token for BERT input
# train_data['bert_input'] = train_data.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))
# test_data['bert_input'] = test_data.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))

<ipython-input-18-ae30c4f648b7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['bert_input'] = train_data.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))
<ipython-input-18-ae30c4f648b7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['bert_input'] = test_data.groupby('episode')['utterances'].transform(lambda x: ' [SEP] '.join(x))


In [ ]:
# Define a function to concatenate utterances within each dialogue
def concatenate_utterances(dialogue_df, sep_token=' [SEP] '):
    concatenated_dialogues = dialogue_df.groupby('episode')['utterances'].apply(lambda x: sep_token.join(x)).reset_index(name='bert_input')
    return concatenated_dialogues

# Apply the function to your train, validation, and test datasets
train_dialogues = concatenate_utterances(train_data)
# val_dialogues = concatenate_utterances(val_data)  # Assuming you have a val_data set up
test_dialogues = concatenate_utterances(test_data)

# Your train_dialogues, val_dialogues, and test_dialogues now have a 'bert_input' column with the concatenated utterances.


In [ ]:
train_dialogues

NameError: name 'train_dialogues' is not defined

In [ ]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [ ]:
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(df, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)

In [ ]:
train_data.shape, val_data.shape,test_data.shape

((3200, 5), (400, 5), (400, 5))

In [ ]:
emotions = train_data['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [ ]:
from transformers import AutoTokenizer

model_card = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_card)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                           num_labels=2,
                                                           problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
for param in model.base_model.parameters():
    print(param)
    param.requires_grad = False

In [ ]:
from datasets import Dataset
training_hf = Dataset.from_pandas(train_data)
validation_hf = Dataset.from_pandas(val_data)
test_hf = Dataset.from_pandas(test_data)

In [ ]:
df['utterances']

0       [also I was the point person on my company's t...
1       [also I was the point person on my company's t...
2       [also I was the point person on my company's t...
3       [also I was the point person on my company's t...
4       [But then who? The waitress I went out with la...
                              ...                        
3995    [Hey., Hey!, So how was Joan?, I broke up with...
3996    [Hey., Hey!, So how was Joan?, I broke up with...
3997    [Hey., Hey!, So how was Joan?, I broke up with...
3998    [Hey., Hey!, So how was Joan?, I broke up with...
3999    [Hey., Hey!, So how was Joan?, I broke up with...
Name: utterances, Length: 4000, dtype: object

In [ ]:
dialogues = df['utterances']
#print(sentences)
max_len = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  if len(dialogue) > max_len:
    max_len = len(dialogue)
    index = idx
max_len,index

(24, 219)

### Be careful for the token used during padding

In [ ]:
triggers = df['triggers']
padded_triggers = [seq + [0] * (max_len - len(seq)) for seq in triggers]

In [ ]:
tokenized_input_ids = []
tokenized_attention_mask = []
tokenized_token_type_ids = []

for utterance_list in df['utterances']:
  utterance_input_ids = []
  utterance_attention_mask = []
  utterance_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    utterance_input_ids.append(tokenized_texts.input_ids)
    utterance_attention_mask.append(tokenized_texts.attention_mask)
    utterance_token_type_ids.append(tokenized_texts.token_type_ids)

  tokenized_input_ids.append(utterance_input_ids)
  tokenized_attention_mask.append(utterance_attention_mask)
  tokenized_token_type_ids.append(utterance_token_type_ids)

In [ ]:
pad = torch.from_numpy(np.array([0]))
pad

tensor([0])

In [ ]:
padded_utterances_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_input_ids]
padded_utterances_attention_mask = [seq + [torch.from_numpy(np.array([1]))] * (max_len - len(seq)) for seq in tokenized_attention_mask]
padded_utterances_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_token_type_ids]

In [ ]:
padded_utterances_input_ids[0]

[tensor([[  101,  2036,  1045,  2001,  1996,  2391,  2711,  2006,  2026,  2194,
           1005,  1055,  6653,  2013,  1996,  1047,  2140,  1011,  1019,  2000,
          24665,  1011,  1020,  2291,  1012,   102]]),
 tensor([[ 101, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012,  102]]),
 tensor([[ 101, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012,  102]]),
 tensor([[ 101, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704,
          1012,  102]]),
 tensor([[ 101, 2026, 5704, 1029, 2035, 2157, 1012,  102]]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0])]

In [ ]:
emotions_input_ids = []
emotions_attention_mask = []
emotions_token_type_ids = []

for utterance_list in df['utterances']:
  emotion_input_ids = []
  emotion_attention_mask = []
  emotion_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    emotion_input_ids.append(tokenized_texts.input_ids)
    emotion_attention_mask.append(tokenized_texts.attention_mask)
    emotion_token_type_ids.append(tokenized_texts.token_type_ids)

  emotions_input_ids.append(emotion_input_ids)
  emotions_attention_mask.append(emotion_attention_mask)
  emotions_token_type_ids.append(emotion_token_type_ids)

In [ ]:
pad_as_1 = torch.from_numpy(np.array([1]))
padded_emotions_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_input_ids]
padded_emotions_attention_mask = [seq + [pad_as_1] * (max_len - len(seq)) for seq in emotions_attention_mask]
padded_emotions_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_token_type_ids]

In [ ]:
padded_emotions[0]

[tensor([[  101,  2036,  1045,  2001,  1996,  2391,  2711,  2006,  2026,  2194,
           1005,  1055,  6653,  2013,  1996,  1047,  2140,  1011,  1019,  2000,
          24665,  1011,  1020,  2291,  1012,   102]]),
 tensor([[ 101, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012,  102]]),
 tensor([[ 101, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012,  102]]),
 tensor([[ 101, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704,
          1012,  102]]),
 tensor([[ 101, 2026, 5704, 1029, 2035, 2157, 1012,  102]]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0])]

In [ ]:
padded_triggers = [seq + [0] * (max_len - len(seq)) for seq in df['triggers']]
padded_triggers[0]

[0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn

len_emotions = len(emotions)

class EmotionTriggerModel(nn.Module):
    def _init_(self, bert_model_name='bert-base-uncased', num_emotions=len_emotions, num_triggers=2):
        super(EmotionTriggerModel, self)._init_()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.emotion_classifier = nn.Linear(self.bert.config.hidden_size, num_emotions)
        self.trigger_classifier = nn.Linear(self.bert.config.hidden_size, num_triggers)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.pooler_output
        emotion_logits = self.emotion_classifier(cls_embedding)
        trigger_logits = self.trigger_classifier(cls_embedding)
        return emotion_logits, trigger_logits

# Example usage:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = EmotionTriggerModel()

#inputs = tokenizer(df['utterances'].tolist(), return_tensors="pt", padding=True, truncation=True)

tokenized_input_ids = []
tokenized_attention_mask = []

for utterance_list in df['utterances']:
  utterance_input_ids = []
  utterance_attention_mask = []
  for sentence in utterance_list:
    tokenized_texts = tokenizer(sentence, padding='max_length', max_length=max_len, return_tensors='pt')
    utterance_input_ids.append(tokenized_texts.input_ids)
    utterance_attention_mask.append(tokenized_texts.attention_mask)

  tokenized_input_ids.append(utterance_input_ids)
  tokenized_attention_mask.append(utterance_attention_mask)

# Now you can use input_ids and attention_mask directly in your model
#emotion_logits, trigger_logits = model(input_ids, attention_mask)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def preprocess_text(texts):
    return tokenizer(texts['utterances'], truncation=True)

training = training_hf.map(preprocess_text, batched=True)
validation = validation_hf.map(preprocess_text, batched=True)
test = test_hf.map(preprocess_text, batched=True)

"""def preprocess_text2(dataset_name):
    tokenized_conclusion = tokenizer(dataset_name['Conclusion'], truncation=False)
    tokenized_premise = tokenizer(dataset_name['Premise'], truncation=False)
    extended_input_ids = [premise + conclusion for premise, conclusion in zip(tokenized_conclusion['input_ids'], tokenized_premise['input_ids'])]
    extended_attention_mask = [premise + conclusion for premise, conclusion in zip(tokenized_conclusion['attention_mask'], tokenized_premise['attention_mask'])]
    extended_token_type_ids = [premise + conclusion for premise, conclusion in zip(tokenized_conclusion['token_type_ids'], tokenized_premise['token_type_ids'])]
    concatenated_dataset = {
        'input_ids': extended_input_ids,
        'attention_mask': extended_attention_mask,
        'token_type_ids': extended_token_type_ids,
    }
    return concatenated_dataset
"""
training_CP = training_hf.map(preprocess_text2, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                           num_labels=4,
                                                           problem_type="multi_label_classification")

In [ ]:
training_CP = training_CP.rename_column('Values', 'labels')
validation_CP = validation_CP.rename_column('Values', 'labels')
test_CP = test_CP.rename_column('Values', 'labels')

In [ ]:
df['emotions'][4]

['surprise', 'sadness', 'surprise', 'fear']

In [ ]:
train_emotions[4]

array([0, 0, 1, 0, 0, 1, 1])

In [ ]:
"""from datasets import Dataset
training_hf = Dataset.from_pandas(train_data)
validation_hf = Dataset.from_pandas(val_data)
test_hf = Dataset.from_pandas(test_data)"""

In [ ]:
"""#old code
def preprocess_text(texts):
    return tokenizer(texts['Conclusion'], truncation=True)

training_C = training_hf.map(preprocess_text, batched=True)
validation_C = validation_hf.map(preprocess_text, batched=True)
test_C = test_hf.map(preprocess_text, batched=True)"""

In [ ]:
train_utterances = [" ".join(utterance) for utterance in train_data['utterances']]
val_utterances = [" ".join(utterance) for utterance in val_data['utterances']]

In [ ]:
# Tokenize the utterances
train_encodings = tokenizer(train_utterances, padding='max_length', max_length=max_len, truncation=True)
val_encodings = tokenizer(val_utterances, padding='max_length', max_length=max_len, truncation=True)

In [ ]:
len(train_encodings.input_ids)

3200

### Nalin's scribbles below

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch
num_labels = len(emotions)
# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

# Initialize MultiLabelBinarizer for emotions
mlb = MultiLabelBinarizer()
mlb.fit(train_data['emotions'])

def preprocess_data(data, tokenizer, mlb):
    # Tokenize the utterances
    tokenized_inputs = tokenizer(
        [" ".join(utterance) for utterance in data['utterances']],
        padding='max_length',
        truncation=True,
        max_length=128, # we might wanna change this to max_len !
        return_tensors='pt'
    )

    # Encode the emotions
    encoded_labels = mlb.transform(data['emotions'])

    # Convert labels to tensors
    labels_tensor = torch.tensor(encoded_labels, dtype=torch.float)

    # Ensure the encoded labels are the correct shape (this step is crucial to avoid the batch size mismatch)
    if labels_tensor.shape[0] != tokenized_inputs['input_ids'].shape[0]:
        raise ValueError(f"Number of examples {tokenized_inputs['input_ids'].shape[0]} does not match number of labels {labels_tensor.shape[0]}")

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

# Now call preprocess_data to process your train and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb)
val_dataset = preprocess_data(val_data, tokenizer, mlb)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])

Single train example: {'input_ids': [101, 2036, 1045, 2001, 1996, 2391, 2711, 2006, 2026, 2194, 1005, 1055, 6653, 2013, 1996, 1047, 2140, 1011, 1019, 2000, 24665, 1011, 1020, 2291, 1012, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704, 1012, 2026, 5704, 1029, 2035, 2157, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Second Preprocessing

In [ ]:
from transformers import BertTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch

def preprocess_data(data, tokenizer, mlb, max_length=512):
    tokenized_inputs = {'input_ids': [], 'attention_mask': []}
    labels = []

    for _, row in data.iterrows():
        utterances = row['utterances']  # assuming this is a list of strings
        emotion_labels = row['emotions']  # assuming this is a list of lists

        # Start with [CLS] token
        tokens = [tokenizer.cls_token_id]
        # Create a zero-vector for [CLS] token's label
        zero_vector = [0] * len(mlb.classes_)
        # Start the label sequence with the label for [CLS] token
        utterance_labels = [zero_vector]

        for utterance, label in zip(utterances, emotion_labels):
            # Encode the utterance to tokens and remove the last [SEP] token
            utterance_tokens = tokenizer.encode(utterance, add_special_tokens=False)
            # Check if adding this utterance exceeds the max_length
            if len(tokens) + len(utterance_tokens) + 1 > max_length:
                break
            # Extend the tokens and labels
            tokens.extend(utterance_tokens + [tokenizer.sep_token_id])
            label_vector = mlb.transform([label])[0]
            utterance_labels.extend([label_vector] * len(utterance_tokens) + [zero_vector])

        # Truncate tokens and labels to max_length
        tokens = tokens[:max_length]
        utterance_labels = utterance_labels[:max_length]

        # The attention mask has 1s where we have tokens and 0s where we have padding
        attention_mask = [1] * len(tokens) + [0] * (max_length - len(tokens))

        # Pad tokens and labels if necessary
        padding_length = max_length - len(tokens)
        tokens.extend([tokenizer.pad_token_id] * padding_length)
        utterance_labels.extend([zero_vector] * padding_length)

        # Append the tokenized input and its corresponding labels to the outputs
        tokenized_inputs['input_ids'].append(tokens)
        tokenized_inputs['attention_mask'].append(attention_mask)
        labels.append(utterance_labels)

    # Convert lists to tensors
    tokenized_inputs = {key: torch.tensor(val, dtype=torch.long) for key, val in tokenized_inputs.items()}
    labels_tensor = torch.tensor(labels, dtype=torch.float)

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

# Initialize the tokenizer and MultiLabelBinarizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)
mlb = MultiLabelBinarizer()
mlb.fit([emotion for sublist in train_data['emotions'] for emotion in sublist])

# Process the datasets
train_dataset = preprocess_data(train_data, tokenizer, mlb, max_length=128)
val_dataset = preprocess_data(val_data, tokenizer, mlb, max_length=128)

# Verify the preprocessed data
print("Single train example:", train_dataset[0])


Single train example: {'input_ids': [101, 2036, 1045, 2001, 1996, 2391, 2711, 2006, 2026, 2194, 1005, 1055, 6653, 2013, 1996, 1047, 2140, 1011, 1019, 2000, 24665, 1011, 1020, 2291, 1012, 102, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012, 102, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012, 102, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704, 1012, 102, 2026, 5704, 1029, 2035, 2157, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Third Preprocessing

In [ ]:
from transformers import BertTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch

def preprocess_data(data, tokenizer, mlb, max_length=512):
    tokenized_inputs = {'input_ids': [], 'attention_mask': []}
    all_labels = []

    for _, row in data.iterrows():
        utterances = row['utterances']
        emotion_labels = row['emotions']

        tokens = [tokenizer.cls_token_id]
        utterance_labels = []
        for utterance, label in zip(utterances, emotion_labels):
            utterance_tokens = tokenizer.encode(utterance, add_special_tokens=False)
            # Ensure we account for space for [SEP] token
            if len(tokens) + len(utterance_tokens) + 1 > max_length:
                break
            tokens.extend(utterance_tokens + [tokenizer.sep_token_id])
            label_vector = mlb.transform([label])[0]
            utterance_labels.extend([label_vector] * len(utterance_tokens) + [label_vector])

        tokens = tokens[:max_length]
        utterance_labels = utterance_labels[:max_length]

        # Inside your loop before padding
        print(f"Length of tokens before padding: {len(tokens)}")
        print(f"Length of utterance_labels before padding: {len(utterance_labels)}")

        padding_length = max_length - len(tokens)
        tokens.extend([tokenizer.pad_token_id] * padding_length)
        attention_mask = [1] * len(tokens) + [0] * padding_length

        zero_vector = [0] * len(mlb.classes_)
        utterance_labels.extend([zero_vector] * padding_length)

        tokenized_inputs['input_ids'].append(tokens)
        tokenized_inputs['attention_mask'].append(attention_mask)
        all_labels.append(utterance_labels)

    tokenized_inputs = {key: torch.tensor(val, dtype=torch.long) for key, val in tokenized_inputs.items()}
    labels_tensor = torch.tensor(all_labels, dtype=torch.float)

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

# Initialize tokenizer and MultiLabelBinarizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)
mlb = MultiLabelBinarizer()
mlb.fit([emotion for sublist in train_data['emotions'] for emotion in sublist])

# Process the datasets
train_dataset = preprocess_data(train_data, tokenizer, mlb, max_length=512)
val_dataset = preprocess_data(val_data, tokenizer, mlb, max_length=512)

# Verify the preprocessed data
print("Single train example:", train_dataset[0])


In [ ]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get the [SEP] token ID
sep_token_id = tokenizer.sep_token_id

print(sep_token_id) # Finding out which token the seperator is assigned to

102


In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3200
})

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply sigmoid to logits and threshold to get multi-label predictions
    sigmoid_logits = 1 / (1 + np.exp(-logits))  # Sigmoid function
    threshold = 0.5
    predictions = (sigmoid_logits > threshold).astype(int)

    # Calculate F1 score for each individual label/class
    f1_scores = [f1_score(labels[:, i], predictions[:, i], average='binary') for i in range(num_labels)]

    # Sequence F1: compute the F1-score for each dialogue and report the average score
    sequence_f1 = np.mean(f1_scores)

    # Unrolled Sequence F1: flatten all utterances and compute the F1-score
    unrolled_labels = labels.flatten()
    unrolled_predictions = predictions.flatten()
    unrolled_sequence_f1 = f1_score(unrolled_labels, unrolled_predictions, average='binary')

    return {
        'sequence_f1': sequence_f1,
        'unrolled_sequence_f1': unrolled_sequence_f1
    }

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
)

# Initialize the model with the number of emotion labels
num_labels = len(mlb.classes_)
model = BertForSequenceClassification.from_pretrained(model_card,
                                                      num_labels=num_labels,
                                                       )


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

# Train the model
trainer.train()
results = trainer.evaluate()

print(results)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

ValueError: Expected input batch_size (2048) to match target batch_size (32768).

In [ ]:
val_dataset
val_dataset['labels']

In [ ]:
from torch.nn.functional import softmax
import numpy as np

# Get raw predictions
raw_predictions = trainer.predict(val_dataset)

# Extract the logits
logits = raw_predictions.predictions

In [ ]:
print(logits)
print(raw_predictions)

[[-0.20233324 -1.5893016  -1.5440171  ...  2.8202894  -0.44170403
   0.33763418]
 [-0.20233324 -1.5893016  -1.5440171  ...  2.8202894  -0.44170403
   0.33763418]
 [-0.20233324 -1.5893016  -1.5440171  ...  2.8202894  -0.44170403
   0.33763418]
 ...
 [-0.34067833 -1.719578   -1.5359057  ...  3.041416   -0.5914837
   0.49952865]
 [-0.3633105  -1.6928407  -1.5281582  ...  3.069526   -0.6153074
   0.5095074 ]
 [-0.28568265 -1.6273079  -1.4687146  ...  3.0057628  -0.55646193
   0.5334556 ]]
PredictionOutput(predictions=array([[-0.20233324, -1.5893016 , -1.5440171 , ...,  2.8202894 ,
        -0.44170403,  0.33763418],
       [-0.20233324, -1.5893016 , -1.5440171 , ...,  2.8202894 ,
        -0.44170403,  0.33763418],
       [-0.20233324, -1.5893016 , -1.5440171 , ...,  2.8202894 ,
        -0.44170403,  0.33763418],
       ...,
       [-0.34067833, -1.719578  , -1.5359057 , ...,  3.041416  ,
        -0.5914837 ,  0.49952865],
       [-0.3633105 , -1.6928407 , -1.5281582 , ...,  3.069526  ,
    

In [ ]:
# Apply softmax to convert logits to probabilities
probabilities = softmax(torch.from_numpy(logits), dim=-1)

# Convert probabilities to numpy array (if needed)
probabilities = probabilities.numpy()

# Now, probabilities holds the prediction probabilities for each class
# For each sample, you can find the class with the highest probability
predicted_classes = np.argmax(probabilities, axis=1)

# Print predicted classes or do further analysis
print(predicted_classes)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [ ]:
emotion_mapping = mlb.classes_
predicted_emotion = emotion_mapping[4]
predicted_emotion

emotion_mapping

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

## For two classifier heads, one common model


In [ ]:
!pip install datasets==2.13.2
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertPreTrainedModel, BertModel
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.nn import BCEWithLogitsLoss, Linear
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset

class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels_emotions, num_labels_triggers):
        super().__init__(config)
        self.num_labels_emotions = num_labels_emotions
        self.num_labels_triggers = num_labels_triggers

        self.bert = BertModel(config)
        self.classifier_emotions = Linear(config.hidden_size, num_labels_emotions)
        self.classifier_triggers = Linear(config.hidden_size, num_labels_triggers)

        # You can initialize weights here if needed

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels_emotions=None,
        labels_triggers=None,
        return_dict=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        pooled_output = outputs[1]

        logits_emotions = self.classifier_emotions(pooled_output)
        logits_triggers = self.classifier_triggers(pooled_output)

        loss = None
        # Calculate loss only if both labels are provided
        if labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        # Adjust the return statement to match what your compute_loss method expects
        if not return_dict:
            outputs = (logits_emotions, logits_triggers)
            if loss is not None:
                outputs = (loss,) + outputs
        else:
            outputs = {'loss': loss, 'logits_emotions': logits_emotions, 'logits_triggers': logits_triggers}

        return outputs
        """
        if not return_dict:
            print(f"Logits Emotions: {logits_emotions.shape}, Logits Triggers: {logits_triggers.shape}")
            output = (logits_emotions, logits_triggers) if loss is None else (loss, logits_emotions, logits_triggers)
            return (loss, logits_emotions, logits_triggers) if loss is not None else (logits_emotions, logits_triggers)
        else:
            raise NotImplementedError("return_dict is set to True, but this model does not support it.")
            """

In [ ]:
# Initialize MultiLabelBinarizers
mlb_emotions = MultiLabelBinarizer()
mlb_triggers = MultiLabelBinarizer()
mlb_emotions.fit(train_data['emotions'])
mlb_triggers.fit(train_data['triggers'])

# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess_data(data, tokenizer, mlb_emotions, mlb_triggers):
    tokenized_inputs = tokenizer(
      [" ".join(utterance) for utterance in data['utterances']],
      padding='max_length',
      truncation=True,
      max_length=128,
      return_tensors='pt'
    )

    # Encode the emotions and triggers
    encoded_labels_emotions = mlb_emotions.transform(data['emotions'])
    encoded_labels_triggers = mlb_triggers.transform(data['triggers'])

    # Convert labels to tensors and reshape
    labels_emotions = torch.tensor(encoded_labels_emotions, dtype=torch.float).view(-1, mlb_emotions.classes_.size)
    labels_triggers = torch.tensor(encoded_labels_triggers, dtype=torch.float).view(-1, mlb_triggers.classes_.size)

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels_emotions': labels_emotions,
        'labels_triggers': labels_triggers
    })

# Then preprocess the training and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb_emotions, mlb_triggers)
val_dataset = preprocess_data(val_data, tokenizer, mlb_emotions, mlb_triggers)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])


Single train example: {'input_ids': [101, 2036, 1045, 2001, 1996, 2391, 2711, 2006, 2026, 2194, 1005, 1055, 6653, 2013, 1996, 1047, 2140, 1011, 1019, 2000, 24665, 1011, 1020, 2291, 1012, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704, 1012, 2026, 5704, 1029, 2035, 2157, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from sklearn.metrics import f1_score

# Define your compute_metrics function to handle both sets of labels
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_emotions = logits[0]
    logits_triggers = logits[1]
    labels_emotions = labels[0]
    labels_triggers = labels[1]
    # Sigmoid function to convert logits to probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    # Threshold to convert probabilities to binary predictions
    threshold = 0.5

    # Convert logits to probabilities
    probs_emotions = sigmoid(logits_emotions)
    probs_triggers = sigmoid(logits_triggers)

    # Convert probabilities to binary predictions
    preds_emotions = (probs_emotions > threshold).astype(int)
    preds_triggers = (probs_triggers > threshold).astype(int)

    # Calculate the f1-score for emotions
    f1_emotions = f1_score(labels_emotions, preds_emotions, average='macro')
    # Calculate the f1-score for triggers
    f1_triggers = f1_score(labels_triggers, preds_triggers, average='macro')

    # To calculate sequence-level metrics, we might need to modify data processing

    # to include sequence IDs, and then group by sequence ID before calculating metrics.
    return {
        'f1_emotions': f1_emotions,
        'f1_triggers': f1_triggers,
    }

In [ ]:
#TODO make this compatible with model class by changing the way the absence of loss is handled, similar to CustomTrainer4

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if len(outputs) == 2:
            logits_emotions, logits_triggers = outputs
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None and labels_triggers is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1, self.model.num_labels_emotions))
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1, self.model.num_labels_triggers))
                loss = loss_emotions + loss_triggers
        elif len(outputs) == 3:
            # Outputs include a loss
            loss, logits_emotions, logits_triggers = outputs
        else:
            raise ValueError(f"Unexpected number of outputs from model: {len(outputs)}")

        # If loss was not computed within the model, we compute it based on the logits and the labels
        if loss is None and labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        return (loss, outputs) if return_outputs else loss


In [ ]:
class CustomTrainer4(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if 'loss' not in outputs:
            logits_emotions = outputs['logits_emotions']
            logits_triggers = outputs['logits_triggers']
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            if labels_triggers is not None:
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers
            outputs['loss'] = loss
        else:
            # Loss is included in outputs
            loss = outputs['loss']

        return (loss, outputs) if return_outputs else loss


In [ ]:
# Initialize model with the number of emotion and trigger labels
num_labels_emotions = len(mlb_emotions.classes_)
num_labels_triggers = len(mlb_triggers.classes_)
model = BertForMultiLabelSequenceClassification.from_pretrained(
    model_card,
    num_labels_emotions=num_labels_emotions,
    num_labels_triggers=num_labels_triggers
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not initialized from the model checkpo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [ ]:
# Assuming train_dataset is a Hugging Face dataset with the correct columns
example = train_dataset[0]

# Now, before passing tensors to the model, ensure they are also on the same device
example_tensors = {k: torch.tensor([v]).to(device) for k, v in example.items()}

# Call the model
with torch.no_grad():
    outputs = model(**example_tensors)
    print(outputs)

ValueError: Target size (torch.Size([7])) must be the same as input size (torch.Size([1, 7]))

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
)

# # Initialize the model with the number of emotion labels
# num_labels = len(mlb.classes_)
# model = BertForSequenceClassification.from_pretrained(model_card,
#                                                       num_labels=num_labels,
#                                                        )

# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

# Train the model
trainer.train()
results = trainer.evaluate()

print(results)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


NameError: name 'f1_score' is not defined